In [1]:
import os
import tempfile

import matplotlib.pyplot as plt
import numpy as np

import pyabc
import pandas as pd

In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [3]:
eng.addpath('scripts/')

'/home/joeleliason/Documents/MATLAB:/usr/local/MATLAB/R2023b/toolbox/matlab/addon_enable_disable_management/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addon_updates/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons:/usr/local/MATLAB/R2023b/toolbox/matlab/addons/cef:/usr/local/MATLAB/R2023b/toolbox/matlab/addons/fileexchange:/usr/local/MATLAB/R2023b/toolbox/matlab/addons/supportpackages:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_common/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_desktop_registration:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_install_location/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_product:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_product_support/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_registry/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_sidepanel/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addressbar_plugins/browse_for_folder_button/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addressbar_plugins/c

In [4]:
param_struct = {
    'k_C1_growth': 0.0160,
    'k_C1_death': 2.0091e-04,
    'k_T1': 0.1108,
    'k_Treg': 0.2179,
    'k_cell_clear': 0.0139,
    'k_C_T1': 1.0133,
    'k_P1_d1': 2.1137e-08,
    'n_T1_clones': 67.0381,
    'n_T0_clones': 296.1959,
    'N_IL2_CD8': 9.7700,
    'N_IL2_CD4': 6.4747,
    'q_T0_T_in': 1.1852e-04,
    'q_T1_T_in': 2.0942e-04,
    'initial_tumour_diameter': 4.2110,
    'T_PD1_total': 1.6162e+04,
    'PD1_50': 24.8685,
    'r_PDL2C1': 0.0011,
    'r_PDL2APC': 0.0307,
    'k_Th_Treg': 0.0043,
    'k_MDSC_rec': 5.0141e+03,
    'k_Mac_rec': 2.6855e+05,
    'k_M1_pol': 0.0149,
    'k_M2_pol': 0.4075,
    'k_CCL2_sec': 2.1667e-12,
    'C_CD47': 63.8422,
    'M_SIRPa': 120.2605,
    'M_PD1_total': 6.5621e+03,
    'k_M1_phago': 0.2636,
    'SIRPa_50': 30.4713,
    'K_Mac_C': 1.0584,
    'q_P_aPDL1': 5.8704e-06,
    'k_cl_aPDL1': 0.3597,
    'gamma_P_aPDL1': 0.0576,
    'V_C': 6.3577,
    'k_cln_aPDL1': 5.1502,
    'Kc_aPDL1': 1.2682
}


In [5]:
# success,M1,M2,Teff,T1exh,T0,Th,Thexh,C_total,VDT = eng.PSA_NSCLC_iteration(param_struct,nargout=10)
M1_M2,Treg_CD8,CD8_CD4 = eng.PSA_NSCLC_iteration(param_struct,nargout=3)

starting sampling
Unable to perform dimensional analysis for rule 'pTCR_p0_MHC_tot = k_M1p0_TCR_off/(k_M1p0_TCR_off+phi_M1p0_TCR)*(k_M1p0_TCR_p/(k_M1p0_TCR_off+k_M1p0_TCR_p))^N_M1p0_TCR*(0.5*(A_s.M1p0/n_T0_clones+TCR_p0_tot+k_M1p0_TCR_off/k_M1p0_TCR_on-TCR_p0_tot*sqrt(((A_s.M1p0/n_T0_clones+TCR_p0_tot+k_M1p0_TCR_off/k_M1p0_TCR_on)/TCR_p0_tot)^2-4*A_s.M1p0/n_T0_clones/TCR_p0_tot)))' because of the function 'sqrt' in the rule. Because UnitConversion is on, correct simulation results will depend on this expression being dimensionally correct. Additionally, SimBiology simulates the model in a unit system determined at runtime. The units are determined by the units used in the model and the model's configset. Unless the inputs and outputs to the function are dimensionless, results may change due to configset option changes, changes to the model, or version changes in SimBiology. It is recommended that input and output arguments to functions be dimensionless to ensure correct results.

> In 

In [6]:
CD8_CD4

2.623875731518125

In [7]:
import pyabc

In [8]:
prior = pyabc.Distribution(mu=pyabc.RV("uniform", 0, 5))

In [9]:
# Define a dictionary to store parameter distributions
prior = {}

# k_C1_growth
prior['k_C1_growth'] = pyabc.RV("loguniform", 1e-5, 5)

# k_C1_death
prior['k_C1_death'] = pyabc.RV("loguniform", 1e-5, 1e-3)

# k_T1
prior['k_T1'] = pyabc.RV("loguniform", 0.01, 1)

# k_Treg
prior['k_Treg'] = pyabc.RV("loguniform", 0.01, 1)

# k_cell_clear
prior['k_cell_clear'] = pyabc.RV("loguniform", 0.01, 1)

# k_C_T1
prior['k_C_T1'] = pyabc.RV("lognorm", 0.9, 1)

# k_P1_d1
prior['k_P1_d1'] = pyabc.RV("lognorm", 40e-9, 1)

# n_T1_clones
prior['n_T1_clones'] = pyabc.RV("lognorm", 92, 1)

# n_T0_clones
prior['n_T0_clones'] = pyabc.RV("lognorm", 100, 1)

# N_IL2_CD8
prior['N_IL2_CD8'] = pyabc.RV("uniform", 9, 13)

# N_IL2_CD4
prior['N_IL2_CD4'] = pyabc.RV("uniform", 5, 11)

# q_T0_T_in
prior['q_T0_T_in'] = pyabc.RV("uniform", 0.000145/2, 0.000145*2)

# q_T1_T_in
prior['q_T1_T_in'] = pyabc.RV("uniform", 0.000145/2, 0.000145*2)

# initial_tumour_diameter
prior['initial_tumour_diameter'] = pyabc.RV("lognorm", 3.7, 0.4)

# T_PD1_total
prior['T_PD1_total'] = pyabc.RV("loguniform", 3.1e3, 1.2e4*20)

# PD1_50
prior['PD1_50'] = pyabc.RV("lognorm", 6, 1)

# r_PDL2C1
prior['r_PDL2C1'] = pyabc.RV("loguniform", 0.001, 0.1)

# r_PDL2APC
prior['r_PDL2APC'] = pyabc.RV("loguniform", 0.001, 0.1)

# k_Th_Treg
prior['k_Th_Treg'] = pyabc.RV("lognorm", 0.022, 1)

# k_MDSC_rec
prior['k_MDSC_rec'] = pyabc.RV("lognorm", 2e3, 0.6)

# k_Mac_rec
prior['k_Mac_rec'] = pyabc.RV("lognorm", 2e5, 0.6)

# k_M1_pol
prior['k_M1_pol'] = pyabc.RV("lognorm", 0.02, 1)

# k_M2_pol
prior['k_M2_pol'] = pyabc.RV("lognorm", 0.25, 1)

# k_CCL2_sec
prior['k_CCL2_sec'] = pyabc.RV("lognorm", 1.7e-12, 0.6)

# C_CD47
prior['C_CD47'] = pyabc.RV("uniform", 50, 150)

# M_SIRPa
prior['M_SIRPa'] = pyabc.RV("uniform", 20, 180)

# M_PD1_total
prior['M_PD1_total'] = pyabc.RV("loguniform", 1.5e3, 3.1e3*20)

# k_M1_phago
prior['k_M1_phago'] = pyabc.RV("lognorm", 0.33, 1)

# SIRPa_50
prior['SIRPa_50'] = pyabc.RV("lognorm", 37, 0.5)

# K_Mac_C
prior['K_Mac_C'] = pyabc.RV("lognorm", 2, 1)


In [10]:
prior = pyabc.Distribution(prior)

In [11]:
def model(parameter):
	parameter = dict(parameter)
	M1_M2,Treg_CD8,CD8_CD4 = eng.PSA_NSCLC_iteration(parameter,nargout=3)

	# return {'M1': M1, 'M2': M2, 'Teff': Teff, 'T1exh': T1exh, 'T0': T0, 'Th': Th, 'Thexh': Thexh, 'C_total': C_total, 'VDT': VDT}
	return {'M1_M2': M1_M2, 'Treg_CD8': Treg_CD8, 'CD8_CD4': CD8_CD4}

In [12]:
observed_data = pd.read_csv('scripts/observed_variables.csv')

In [13]:
observed_data.columns = ['M1_M2', 'Treg_CD8', 'CD8_CD4']

In [14]:
def distance(x, y):
	return min(sum((x[k] - y[k])**2 for k in x.keys()))

In [15]:
eng.addpath('scripts/')

'/home/joeleliason/.MathWorks/SimBiology/4cabcff8-fa36-4ff3-8176-04f2eccb8fb3:/home/joeleliason/Dropbox (University of Michigan)/Projects/forked/QSPIO-TNBC/scripts:/home/joeleliason/Documents/MATLAB:/usr/local/MATLAB/R2023b/toolbox/matlab/addon_enable_disable_management/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addon_updates/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons:/usr/local/MATLAB/R2023b/toolbox/matlab/addons/cef:/usr/local/MATLAB/R2023b/toolbox/matlab/addons/fileexchange:/usr/local/MATLAB/R2023b/toolbox/matlab/addons/supportpackages:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_common/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_desktop_registration:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_install_location/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_product:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_product_support/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_registry/matlab:/usr/local/MATLAB/R2023b/toolbox/matlab/addons_sidep

In [19]:
sample = prior.rvs()
output = model(sample)

starting sampling
Unable to perform dimensional analysis for rule 'pTCR_p0_MHC_tot = k_M1p0_TCR_off/(k_M1p0_TCR_off+phi_M1p0_TCR)*(k_M1p0_TCR_p/(k_M1p0_TCR_off+k_M1p0_TCR_p))^N_M1p0_TCR*(0.5*(A_s.M1p0/n_T0_clones+TCR_p0_tot+k_M1p0_TCR_off/k_M1p0_TCR_on-TCR_p0_tot*sqrt(((A_s.M1p0/n_T0_clones+TCR_p0_tot+k_M1p0_TCR_off/k_M1p0_TCR_on)/TCR_p0_tot)^2-4*A_s.M1p0/n_T0_clones/TCR_p0_tot)))' because of the function 'sqrt' in the rule. Because UnitConversion is on, correct simulation results will depend on this expression being dimensionally correct. Additionally, SimBiology simulates the model in a unit system determined at runtime. The units are determined by the units used in the model and the model's configset. Unless the inputs and outputs to the function are dimensionless, results may change due to configset option changes, changes to the model, or version changes in SimBiology. It is recommended that input and output arguments to functions be dimensionless to ensure correct results.

> In 

Output argument "M1_M2" (and possibly others) not assigned a value in the execution with "PSA_NSCLC_iteration" function.



MatlabExecutionError: Output argument "M1_M2" (and possibly others) not assigned a value in the execution with "PSA_NSCLC_iteration" function.


In [20]:
output

NameError: name 'output' is not defined

In [76]:
dict(sample)

{'k_C1_growth': 0.0001647232231711997,
 'k_C1_death': 0.0001029774857032533,
 'k_T1': 0.02872638579714039,
 'k_Treg': 0.04466055072310252,
 'k_cell_clear': 0.051413280409394045,
 'k_C_T1': 2.458597514271762,
 'k_P1_d1': 1.9999999767727688,
 'n_T1_clones': 7.715102598676841,
 'n_T0_clones': 1.5961426353064412,
 'N_IL2_CD8': 14.575073882943379,
 'N_IL2_CD4': 14.207631618478613,
 'q_T0_T_in': 0.0001172071366098128,
 'q_T1_T_in': 0.0002816494456300048,
 'initial_tumour_diameter': 3.4304601978472884,
 'T_PD1_total': 60378.57952620127,
 'PD1_50': 1.0000569363698473,
 'r_PDL2C1': 0.04515733145692132,
 'r_PDL2APC': 0.0011389841614826092,
 'k_Th_Treg': 1.9971772307821405,
 'k_MDSC_rec': 0.6,
 'k_Mac_rec': inf,
 'k_M1_pol': 2.0098881210387276,
 'k_M2_pol': 1.9208892084616003,
 'k_CCL2_sec': 1.599999999998642,
 'C_CD47': 116.62704940957579,
 'M_SIRPa': 138.71177755658158,
 'M_PD1_total': 3067.6424251210306,
 'k_M1_phago': 2.358937150234622,
 'SIRPa_50': 0.5000002002797845,
 'K_Mac_C': 293.6089984

In [90]:
distance(output,observed_data)

inf

In [91]:
abc = pyabc.ABCSMC(model, prior, distance, population_size=3)

ABC.Sampler INFO: Parallelize sampling on 4 processes.


In [92]:
db_path = os.path.join(tempfile.gettempdir(), "test.db")
observation = observed_data
abc.new("sqlite:///" + db_path, observation)

/home/joeleliason/miniforge3/envs/matlab_QSP/lib/python3.11/site-packages/pyabc/storage/bytes_storage.py:37: UserWarning: Can't find pyarrow, falling back to less efficient csv to store pandas DataFrames.
Install e.g. via `pip install pyabc[pyarrow]`
  return df_to_bytes(object_)
ABC.History INFO: Start <ABCSMC id=4, start_time=2024-04-14 22:02:39>


In [93]:
history = abc.run(max_nr_populations=1, minimum_epsilon=0)

ABC INFO: Calibration sample t = -1.


ABC.History INFO: Done <ABCSMC id=4, duration=0:07:53.787060, end_time=2024-04-14 22:10:33>


KeyboardInterrupt: 

In [ ]:
# execution_time = timeit.timeit(lambda: eng.PSA_NSCLC_iteration(1,nargout=10), number=6)
# print("Execution time:", execution_time, "seconds")